<h1 style="border:2px solid Purple;text-align:center">RANZCR CLiP - Catheter and Line Position Challenge</h1>

![](https://storage.googleapis.com/kaggle-organizations/3765/thumbnail.png?r=270)

<h1 style="border:2px solid Purple;text-align:center">Table of Contents</h1>

* [The Competition](#competition)

* [Objective](#obj)

* [Dataset](#ds)

* [Importing Necessary Libraries](#libs)

* [Training Dataset](#trainds)

* [Image Annotations](#annot)

* [Image Information](#image)

* [Category Overlap](#overlap)

<a id = "competition"></a>

<h1 style="border:2px solid Purple;text-align:center">The Competition</h1>

Serious complications can occur as a result of malpositioned lines and tubes in patients. Doctors and nurses frequently use checklists for placement of lifesaving equipment to ensure they follow protocol in managing patients. Yet, these steps can be time consuming and are still prone to human error, especially in stressful situations when hospitals are at capacity.

Hospital patients can have catheters and lines inserted during the course of their admission and serious complications can arise if they are positioned incorrectly. Nasogastric tube malpositioning into the airways has been reported in up to 3% of cases, with up to 40% of these cases demonstrating complications [1-3]. Airway tube malposition in adult patients intubated outside the operating room is seen in up to 25% of cases [4,5]. The likelihood of complication is directly related to both the experience level and specialty of the proceduralist. Early recognition of malpositioned tubes is the key to preventing risky complications (even death), even more so now that millions of COVID-19 patients are in need of these tubes and lines.

The gold standard for the confirmation of line and tube positions are chest radiographs. However, a physician or radiologist must manually check these chest x-rays to verify that the lines and tubes are in the optimal position. Not only does this leave room for human error, but delays are also common as radiologists can be busy reporting other scans. Deep learning algorithms may be able to automatically detect malpositioned catheters and lines. Once alerted, clinicians can reposition or remove them to avoid life-threatening complications.

The Royal Australian and New Zealand College of Radiologists (RANZCR) is a not-for-profit professional organisation for clinical radiologists and radiation oncologists in Australia, New Zealand, and Singapore. The group is one of many medical organisations around the world (including the NHS) that recognizes malpositioned tubes and lines as preventable. RANZCR is helping design safety systems where such errors will be caught.

<a id = "obj"></a>

<h1 style="border:2px solid Purple;text-align:center">Objective</h1>

In this competition, you’ll detect the presence and position of catheters and lines on chest x-rays. Use machine learning to train and test your model on 40,000 images to categorize a tube that is poorly placed.


<a id = "eval"></a>

<h1 style="border:2px solid Purple;text-align:center">Evaluation Metric</h1>

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

To calculate the final score, AUC is calculated for each of the 11 labels, then averaged. The score is then the average of the individual AUCs of each predicted column.

In case you need a python code [Source](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/203581#1115260)

> np.mean([roc_auc_score(y_true[:, i], y_pred[:, i]) for i in range(11)])

<a id = "ds"></a>

<h1 style="border:2px solid Purple;text-align:center">Dataset</h1>

The dataset has been labelled with a set of definitions to ensure consistency with labelling. The normal category includes lines that were appropriately positioned and did not require repositioning. The borderline category includes lines that would ideally require some repositioning but would in most cases still function adequately in their current position. The abnormal category included lines that required immediate repositioning.

<a id = "libs"></a>
<h1 style="border:2px solid Purple;text-align:center">Importing Necessary Libraries</h1>

In [ ]:
import pandas as pd
import numpy as np
import cv2
from ast import literal_eval
from tqdm import tqdm
import os

import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib_venn import *

from plotly.offline import init_notebook_mode
init_notebook_mode()

<a id = "trainds"></a>
<h1 style="border:2px solid Purple;text-align:center">Training Dataset</h1>

In [ ]:
BASE_DIR = "../input/ranzcr-clip-catheter-line-classification/"
df = pd.read_csv(BASE_DIR + 'train.csv')
df.head()

In [ ]:
print(f"The size of the dataset is {df.shape[0]} and it contains {df['PatientID'].nunique()} patient ids")

### Columns

* **StudyInstanceUID** - unique ID for each image
* **ETT - Abnormal** - endotracheal tube placement abnormal
* **ETT - Borderline** - endotracheal tube placement borderline abnormal
* **ETT - Normal** - endotracheal tube placement normal
* **NGT - Abnormal** - nasogastric tube placement abnormal
* **NGT - Borderline** - nasogastric tube placement borderline abnormal
* **NGT - Incompletely Imaged** - nasogastric tube placement inconclusive due to imaging
* **NGT - Normal** - nasogastric tube placement borderline normal
* **CVC - Abnormal** - central venous catheter placement abnormal
* **CVC - Borderline** - central venous catheter placement borderline abnormal
* **CVC - Normal** - central venous catheter placement normal
* **Swan Ganz Catheter Present**
* **PatientID** - unique ID for each patient in the dataset

In [ ]:
idCounts = df['PatientID'].value_counts().reset_index()
idCounts.columns = ['PatientID', 'Number of Observations']
idCounts = idCounts.sort_values(by = 'Number of Observations', ascending = False)
idCounts.head()

In [ ]:
fig = px.histogram(idCounts, 'Number of Observations', title = 'Distribution of Number of Observations per PatientIDs', template = 'ggplot2')
fig.show()

Most of the PatientID's have only one observation in the dataset, but there are some PatientIDs with as many as 100+ records in the training dataset. 

In [ ]:
categories = ['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal','Swan Ganz Catheter Present']
categoryCounts = df[categories].sum(axis = 0).reset_index()
categoryCounts.columns = ['Malpositions', 'Number of Observations']

In [ ]:
fig = px.bar(categoryCounts, y = 'Malpositions', x = 'Number of Observations', template = 'seaborn', text = 'Number of Observations', title = 'Line and tube positions')
fig.show()

In [ ]:
countsDf = {'Type' : [], 'Malposition'  : [], 'Num of Observations' : []}
for Type in ['Normal','Abnormal','Borderline']:
    for malposition in ['ETT','NGT','CVC']:
        colName = f'{malposition} - {Type}'
        countsDf['Type'].append(Type)
        countsDf['Malposition'].append(malposition)
        val = df[colName].sum(axis = 0)
        countsDf['Num of Observations'].append(val)
countsDf = pd.DataFrame(countsDf)

In [ ]:
fig = px.bar(countsDf, x = 'Num of Observations', y = 'Type', color = 'Malposition', barmode = 'stack', 
             color_discrete_map={'ETT' : '#a2885e', 'NGT' : '#e9cf87', 'CVC' : '#f1efd9'}, template = 'plotly_dark')
fig.show()

<a id = "annot"></a>
<h1 style="border:2px solid Purple;text-align:center">Image Annotations</h1>

In [ ]:
annotations = pd.read_csv(BASE_DIR + 'train_annotations.csv')
annotations['data'] = annotations['data'].apply(literal_eval)
annotations.head()

In [ ]:
IMAGE_DIR_TRAIN = BASE_DIR + 'train/'
IMAGE_DIR_TEST = BASE_DIR + 'test/'

In [ ]:
def plot_image_and_annotations(image_uid, title,singleLabel = True):
    image_path = IMAGE_DIR_TRAIN + image_uid + '.jpg'
    data = annotations[annotations['StudyInstanceUID'] == image_uid]
    if(singleLabel):
        data = data[data['label'] == title]
    data = data['data']
    if(len(data) == 0):
        print(title)
        return
    plt.figure(figsize=(10,6))

    plt.subplot(1, 2,1)
    img = plt.imread(image_path)
    
    print(f"Image dimensions:  {img.shape[0],img.shape[1]}")
    print(f"Maximum pixel value : {img.max():.1f} ; Minimum pixel value:{img.min():.1f}")
    print(f"Mean value of the pixels : {img.mean():.1f} ; Standard deviation : {img.std():.1f}")
    
    plt.imshow(img, cmap='gray')
    plt.title('Actual Image')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
    for i in range(len(data)):
        curr_data = data.values[i]
        x_loc = [x[0] for x in curr_data]
        y_loc = [x[1] for x in curr_data]
        
        plt.plot(x_loc, y_loc, linewidth = 5.0)
    plt.tight_layout()
    
    plt.title('Annotated Image')
    
    plt.suptitle(title)
    plt.show()

In [ ]:
imageIds = {}
for cat in categories:
    imageIds[cat] = df[df[cat] == 1]['StudyInstanceUID'].to_list()

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - ETT - Abnormal</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[0]][0], categories[0])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - ETT - Borderline</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[1]][2], categories[1])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - ETT - Normal</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[2]][-1], categories[2])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - NGT - Abnormal</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[3]][-1], categories[3])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - NGT - Borderline</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[4]][-1], categories[4])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - NGT - Incompletely Imaged</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[5]][-1], categories[5])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - NGT - Normal</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[6]][7], categories[6])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - CVC - Abnormal</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[7]][-1], categories[7])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - CVC - Borderline</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[8]][0], categories[8])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - CVC - Normal</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[9]][-1], categories[9])

<h1 style="border:2px solid Purple;text-align:center">Image Annotations - Swan Ganz Catheter Present</h1>

In [ ]:
plot_image_and_annotations(imageIds[categories[10]][5], categories[10])

<a id="image"></a>
<h1 style="border:2px solid Purple;text-align:center">Image Info</h1>

In [ ]:
image_width = []
image_height = []
image_channel_mean = []
for image_uid in tqdm(df['StudyInstanceUID'].to_list()):
    image_path = IMAGE_DIR_TRAIN + image_uid + '.jpg'
    img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    image_height.append(img.shape[0])
    image_width.append(img.shape[1])
    image_channel_mean.append(np.mean(img))
df['Image Height'] = image_height
df['Image Width'] = image_width
df['Total Pixels'] = df['Image Height'] * df['Image Width']

In [ ]:
test_image_width = []
test_image_height = []
test_channel_mean = []
for image_name in tqdm(os.listdir(IMAGE_DIR_TEST)):
    image_path = IMAGE_DIR_TEST + image_name
    img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    test_image_height.append(img.shape[0])
    test_image_width.append(img.shape[1])
    test_channel_mean.append(np.mean(img))

In [ ]:
trace0 = go.Violin(x = image_height, name = 'Train Height')
trace1 = go.Violin(x = image_width, name = 'Train Width')

trace2 = go.Violin(x = test_image_width, name = 'Test Width')
trace3 = go.Violin(x = test_image_height, name = 'Test Height')

fig = go.Figure([trace0,trace3, trace1, trace2])
fig.update_layout(title = 'Image Height and Widths', template = 'ggplot2')
fig.show()

In [ ]:
fig = ff.create_distplot([image_channel_mean, test_channel_mean], group_labels=['Training Images', 'Testing Images'], colors = ['#cecece','#808080'])
fig.update_layout(template = 'plotly_white', title_text = 'Channel Distribution')
fig.show()

<a id="overlap"></a>
<h1 style="border:2px solid Purple;text-align:center">Category Overlap</h1>

In [ ]:
plt.figure(figsize=(6,6))
fig = venn3([set(df[df[col] == 1]['StudyInstanceUID']) for col in ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal']], set_labels = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal'])
plt.title('ETT Image Overlap')
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
fig = venn3([set(df[df[col] == 1]['StudyInstanceUID']) for col in ['NGT - Abnormal', 'NGT - Borderline','NGT - Normal']], set_labels = ['NGT - Abnormal', 'NGT - Borderline','NGT - Normal'])
plt.title('NGT Image Overlap')
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
fig = venn3([set(df[df[col] == 1]['StudyInstanceUID']) for col in ['CVC - Abnormal', 'CVC - Borderline','CVC - Normal']], set_labels = ['CVC - Abnormal', 'CVC - Borderline','CVC - Normal'])
plt.title('CVC Image Overlap')
plt.show()

### It must be noted that that the fact that we have an overlap in NGT and CVC categories does not mean that the images are mislabeled. 

> The labels reflect whether or not a particular finding is present or absent in the image, and are not mutually exclusive as there may be more than one line or tube of that type in the patient. For instance, it is not uncommon to have 2 or more CVCs in the same patient, where one could be in a Normal position but another could be in a Borderline or Abnormal position. This will explain the majority of the cases you see where there are more than one label for NGT or CVC. There may be a small minority of incorrectly labelled cases due to human error but this should be a small percentage.

Read the following [discussion](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/203372) to know more

In [ ]:
df[(df['CVC - Abnormal'] == 1) & (df['CVC - Normal'] == 1) & (df['CVC - Borderline'] == 1)]

In [ ]:
plot_image_and_annotations('1.2.826.0.1.3680043.8.498.74579763900891627023142390145044977755', 'Image with CVC Normal, Abnormal and Borderline Annotations',singleLabel=False)

## Work in Progress